<img width="400" src="https://nomeroff.net.ua/images/logo/nomeroff_net.svg" alt="Nomeroff Net. Automatic numberplate recognition system" align="left"/>

## The example demonstrates license plate number detection.

Before runing this demo, please download models from [https://nomeroff.net.ua/models/](https://nomeroff.net.ua/models/) to **./models/** directory. 

In [1]:
#Specify device
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
import matplotlib.image as mpimg
import os
import pandas as pd

In [3]:
# Import all necessary libraries.
import numpy as np
import sys
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import copy

# nomeroff_net path
NOMEROFF_NET_DIR = os.path.abspath('../../../')
sys.path.append(NOMEROFF_NET_DIR)

In [4]:
from nomeroff_net.YoloV5Detector import Detector

detector = Detector()
detector.load()

In [5]:
from nomeroff_net.bbox_np_points import np_points_craft, get_cv_zone_rgb, convert_cv_zones_rgb_to_bgr, reshape_points

np_points_craft = np_points_craft()
np_points_craft.load()

Loading weights from checkpoint (/mnt/store/nomeroff-net-dev/nomeroff_net/tools/../../data/./models/np_points_craft/craft_mlt/craft_mlt_25k_2020-02-16.pth)
Loading weights of refiner from checkpoint (/mnt/store/nomeroff-net-dev/nomeroff_net/tools/../../data/./models/np_points_craft/craft_refiner/craft_refiner_CTW1500_2020-02-16.pth)


In [8]:
%matplotlib inline
plt.rcParams["figure.figsize"] = (20,10)

In [9]:
root_dir = '/mnt/storage1/datasets/numberplate/eu_dataset_photos_test/'

photos = pd.read_csv('photos_eu_dataset.csv')
photos = photos.set_index(['photoId'])

In [10]:
for dir_name, subdir_list, file_list in os.walk(root_dir):
    for fname in file_list:
        try:
            baseName = os.path.splitext(os.path.basename(fname))[0]
            img_path = os.path.join(dir_name, fname)
            print(img_path)
            img = mpimg.imread(img_path)   

            target_boxes = detector.detect_bbox(copy.deepcopy(img))
            

            all_points = np_points_craft.detect(img, target_boxes)
            all_points = [ps for ps in all_points if len(ps)]

            # cut zones
            to_show_zones = [get_cv_zone_rgb(img, reshape_points(rect, 1)) for rect in all_points]
            zones = convert_cv_zones_rgb_to_bgr(to_show_zones)

            if len(to_show_zones) > 1:
                areas = [zone.shape[0]*zone.shape[1] for zone in zones]
                zone = to_show_zones[np.argmax(areas)]
            else:
                zone = to_show_zones[0]

            text = photos.loc[int(baseName)]['npText']
            imgpath = "/mnt/storage1/datasets/numberplate/eu_numberplates_test/{}_{}.png".format(baseName, text)
            mpimg.imsave(imgpath, zone)

        except Exception as ex:
            print('[ERROR]', ex)

/mnt/storage1/datasets/numberplate/eu_dataset_photos_test/412259506.jpeg
rotate 0.39914163090128757
propably_points
[[832.2904993163336, 681.8512623275226], [823.0, 626.5506133825558], [1046.7095172244713, 588.9674147167312], [1056.0, 644.2680655247069]]
/mnt/storage1/datasets/numberplate/eu_dataset_photos_test/412176490.jpeg
rotate 1.0169491525423728
propably_points
[[128.4987229795356, 693.5548753112739], [148.17117030239427, 656.66904897868], [270.94349670410156, 722.1476211547852], [251.27105712890622, 759.033447265625]]


/mnt/store/nomeroff-net-dev/nomeroff_net/tools/../../data/./repos/np_points_craft/craft_mlt/craft_utils.py:239: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  polys = np.array(polys)
